# Imports.

In [176]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px

# Initial data preprocessing.

In [177]:
events_data = pd.read_csv("D:\Programming\Sheets\event_data_train.csv")
submissions_data = pd.read_csv("D:\Programming\Sheets\submissions_data_train.csv")

In [178]:
events_data.head()

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


In [179]:
submissions_data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [183]:
# Date
events_data['date'] = pd.to_datetime(events_data.timestamp, unit='s')
events_data['day'] = events_data.date.dt.date

submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit='s')
submissions_data['day'] = submissions_data.date.dt.date

# corrects and wrongs
users_scores = submissions_data.pivot_table(index='user_id',
                                            columns='submission_status',
                                            values='step_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()


# Is gone user
users_data = events_data.groupby('user_id', as_index=False) \
    .agg({'timestamp': 'max'}).rename(columns={
        'timestamp': 'last_timestamp'
    })

now = 1526772811
drop_out_treshold = 30 * 24 * 60 * 60
users_data['is_gone_user'] = (
    now - users_data.last_timestamp) > drop_out_treshold

users_data = users_data.merge(users_scores, on='user_id', how='outer')
users_data = users_data.fillna(0)


# Min timestamp
users_min_timestamps = events_data.groupby("user_id", as_index=False).timestamp.min(
).rename(columns={"timestamp": "min_timestamp"})
users_data = users_data.merge(users_min_timestamps, on="user_id", how="outer")


# Users events
users_events_data = events_data.pivot_table(index='user_id',
                                            columns='action',
                                            values='step_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()
users_data = users_data.merge(users_events_data, how='outer')


# Days
users_days = events_data.groupby(
    'user_id').day.nunique().to_frame().reset_index().rename(columns={"day": "days"})
users_data = users_data.merge(users_days, how='outer')


# Passed course
users_data['passed_course'] = users_data.passed > 175


# Print
users_data.head()


,user_id,last_timestamp,is_gone_user,correct,wrong,min_timestamp,discovered,passed,started_attempt,viewed,days,passed_course
0,1,1472827464,True,0.0,0.0,1472827464,1,0,0,1,1,False
1,2,1519226966,True,2.0,0.0,1514383364,9,9,2,10,2,False
2,3,1444581588,True,29.0,23.0,1434358476,91,87,30,192,7,False
3,5,1499859939,True,2.0,2.0,1466156809,11,11,4,12,2,False
4,7,1521634660,True,0.0,0.0,1521634660,1,1,0,1,1,False


Check that the data is not lost.

In [181]:
events_data.user_id.nunique() == users_data.user_id.nunique()

True

# Data preprocessing for ML. 

We will make data for the first three days by which we will use.

In [185]:
# event_data_train
event_data_train = events_data.merge(users_data[['user_id', 'min_timestamp']], on='user_id', how='left') \
    .query("(timestamp - min_timestamp) < (3 * 24 * 60 * 60)")

event_data_train.head()

,step_id,timestamp,action,user_id,date,day,min_timestamp
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,1434340848
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,1434340848


In [186]:
# submission_data_train
submissions_data_train = submissions_data.merge(users_data[['user_id', 'min_timestamp']], on='user_id', how='left') \
    .query("(timestamp - min_timestamp) < (3 * 24 * 60 * 60)")

submissions_data_train.head()

,step_id,timestamp,submission_status,user_id,date,day,min_timestamp
0,31971,1434349275,correct,15853,2015-06-15 06:21:15,2015-06-15,1434345977
1,31972,1434348300,correct,15853,2015-06-15 06:05:00,2015-06-15,1434345977
4,31976,1434348123,wrong,15853,2015-06-15 06:02:03,2015-06-15,1434345977
5,31976,1434348188,correct,15853,2015-06-15 06:03:08,2015-06-15,1434345977
7,31977,1434347371,correct,15853,2015-06-15 05:49:31,2015-06-15,1434345977


## X. 

Let's make X. 

In [230]:
# Unique days.
X = submissions_data_train.groupby("user_id", as_index=False).agg(
    {"day": "nunique"}).rename(columns={"day": "days"})


# Steps tried.
steps_tried = submissions_data_train.groupby(
    "user_id", as_index=False).agg({"step_id": "nunique"}).rename(columns={"step_id": "steps_tried"})

X = X.merge(steps_tried, on="user_id", how="outer")


# Corrects and wrongs.
X = X.merge(submissions_data_train.pivot_table(index="user_id",
                                                     columns="submission_status",
                                                     values="step_id",
                                                     aggfunc="count", fill_value=0).reset_index(), on="user_id", how="outer")


# Correct ratio.
X["correct_ratio"] = X.correct / (X.correct + X.wrong)


# Steps viewed.
steps_viewed = event_data_train.pivot_table(index="user_id",
                                            columns="action",
                                            values="step_id",
                                            aggfunc="count", fill_value=0).reset_index()[["user_id", "viewed"]]

X = X.merge(steps_viewed, on="user_id", how="outer")

X = X.fillna(0)  # we use it, because we used how="outer"


# Add passed_course and is_gone_user
X = X.merge(users_data[["user_id", "passed_course",
            "is_gone_user"]], on="user_id", how="outer")


# Print.
X.head()


,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_course,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [233]:
# Take people who either underwent a course or abandoned it.
X = X.query("passed_course == True | is_gone_user == True")
X.head()


,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_course,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True
